# Multivariate association with MASH
This note book document the process of meta analysis using mash. 

The output of association scan, and optionally the output from METAL, will undergo a reformmating process, and then fed into the MASH module to generate
1. MASH result in vcf format
2. Mixture prior for MvSuSIE_RSS

# Data Processing

## Allele flip QC
1. feed a table of sumstat to yml_generator
2. yml_generator will provide the yml template for allele flip qc
3. The output sumstat_list can direct the next step

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/misc/yml_generator.ipynb yml_list \
--sumstat_list_path /mnt/mfs/statgen/xqtl_workflow_testing/success_example/testing_no_cov/QTL_association/sumstat_list  \
--wd ./  &

nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/misc/summary_stats_merger.ipynb \
--yml_list  /mnt/mfs/statgen/xqtl_workflow_testing/module/Merge/yml_list.txt  \
--cwd ./  &

## Merging and partitioning
This step takes in the qced sumstat_list from the previouse step and generate a merged rds for each gene. To feed into downstream analysis 

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/multivariate/MASH/sumstat_processing.ipynb processing \
--sumstat_list_path /mnt/mfs/statgen/xqtl_workflow_testing/module/Merge/qc_sumstat_list.txt \
--region_list /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_region_long \
--wd ./  --container /mnt/mfs/statgen/containers/xqtl_pipeline_sif/base-bioinfo.sif -s force &

## Extract effect
The stronges, the random, and the null signal will be extracted for each gene, and merged into 1 rds for downstream analysis

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/multivariate/MASH/Signal_Extraction.ipynb extract_effects \
            --cwd ./ \
            --container /mnt/mfs/statgen/containers/twas_latest.sif \
            --name  "PCC_DLPFC_AC" \
            --analysis_units /mnt/mfs/statgen/xqtl_workflow_testing/module/Merge/merge/RDS/PCC_DLPFC_AC.analysis_unit  -s force &

# Mash Analysis

## Construct of MASH Model

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/multivariate/MASH/mashr.ipynb mash \
            --cwd ./ \
            --container /mnt/mfs/statgen/containers/twas_latest.sif \
            --output_prefix "PCC_DLPFC_AC" \
            --data /mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC.rds -s force &

## MASH Posterior
Run the following command in the same working directory as the previouse step

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/multivariate/MASH/mashr.ipynb posterior \
            --cwd ./ \
            --container /mnt/mfs/statgen/containers/twas_latest.sif \
            --output_prefix "PCC_DLPFC_AC" \
            --analysis_units /mnt/mfs/statgen/xqtl_workflow_testing/module/Merge/merge/RDS/PCC_DLPFC_AC.analysis_unit  &

## Convert MASH result RDS to VCF
Running the follwoing code using the mash_output_list from the previouse step as input can covert all the mash RDS into vcf format

In [ ]:
        nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/misc/rds_to_vcf.ipynb rds_to_vcf \
            --wd ./ \
            --container /mnt/mfs/statgen/containers/twas_latest.sif \
            --name "PCC_DLPFC_AC" \
            --analysis_units   /mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/mash_output_list